In [1]:
! pip install torchtext
! pip install spacy
! python3 -m spacy download en

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Defaulting to user installation because normal site-packages is not writeable
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/Users/tingfengx/Library/Python/3.7/lib/python/site-packages/en_core_web_sm -->
/Users/tingfengx/Library/Python/3.7/lib/python/site-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [2]:
# load pytorch
import torch

device = torch.device(
    "cuda:0" 
    if torch.cuda.is_available() 
    else "cpu"
)

from torchtext import data
from torchtext import datasets

RAND_SEED = 1234

torch.manual_seed(RAND_SEED)
torch.backends.cudnn.deterministic = True

TEXT = data.Field(tokenize = 'spacy', include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)

In [3]:
# use the IMDb dataset as a first example
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)
# mae a validation set from training set
import random
train_data, valid_data = train_data.split(
    random_state = random.seed(RAND_SEED))

In [4]:
# use the GLoVe to initialize the word embeddings
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(
    train_data, 
    max_size = MAX_VOCAB_SIZE, 
    vectors = "glove.6B.100d", 
    unk_init = torch.Tensor.normal_
)

LABEL.build_vocab(train_data)

In [5]:
BATCH_SIZE = 64

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_within_batch = True,
    device = device
)

In [6]:
# build the LSTM model

import torch.nn as nn

class SentiLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, 
                 hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):

        super().__init__()
        # make the embeddings
        self.embedding = nn.Embedding(vocab_size, 
                                      embedding_dim,
                                      padding_idx=pad_idx)
        
        self.lstm = nn.LSTM(embedding_dim,
                            hidden_size=hidden_dim,
                            num_layers=n_layers,
                            bidirectional=bidirectional,
                            dropout=dropout)
        
        self.mlp = nn.Linear(hidden_dim * 2, output_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, text, text_lengths):
        # text = [sent len, batch size]
        embedded = self.dropout(self.embedding(text))
        # embedded = [sent len, batch size, emb dim]
        # pack sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths)
        packed_output, (hidden, cell) = self.lstm(packed_embedded)
        # unpack sequence
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)

        # output = [sent len, batch size, hid dim * num directions]
        # output over padding tokens are zero tensors
        
        # hidden = [num layers * num directions, batch size, hid dim]
        # cell = [num layers * num directions, batch size, hid dim]
        
        # concat the final forward (hidden[-2,:,:]) and backward (hidden[-1,:,:]) hidden layers and apply dropout
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))     
        #h idden = [batch size, hid dim * num directions]
            
        return self.mlp(hidden)

In [7]:
# ================= ⬇️ MODEL HYPER PARAMETERS ⬇️ =================
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
LR = 3e-3
N_EPOCHS = 10
# TODO: Add dropout here later
DROPOUT = 0.5
# ================= ⬆️ MODEL HYPER PARAMETERS ⬆️ =================


PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = SentiLSTM(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, 
                  OUTPUT_DIM, N_LAYERS, BIDIRECTIONAL, 
                  DROPOUT, PAD_IDX)

In [8]:
print(model.parameters)

print(i for i in model.parameters())

count_parameters = lambda model: sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

<bound method Module.parameters of SentiLSTM(
  (embedding): Embedding(25002, 100, padding_idx=1)
  (lstm): LSTM(100, 256, num_layers=2, bidirectional=True)
  (mlp): Linear(in_features=512, out_features=1, bias=True)
  (dropout): Dropout(p=0.0, inplace=False)
)>
<generator object <genexpr> at 0x15c54e390>
The model has 4,810,857 trainable parameters


In [9]:
# use the AdamW optimizer
import torch.optim as optim
optimizer = optim.AdamW(model.parameters(), lr=LR)
# loss 
criterion = nn.BCEWithLogitsLoss()

# make sure to use GPU if one is available
model = model.to(device)
criterion = criterion.to(device)

In [10]:
def binary_accuracy(preds, truth):
    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == truth).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [11]:
# the train functio
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        text, text_lengths = batch.text
        
        predictions = model(text, text_lengths).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [12]:
# eval function
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text, text_lengths = batch.text
            
            predictions = model(text, text_lengths).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [ ]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

# initial best loss, inf
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')